In [1]:
from langchain.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from openai import AzureOpenAI
import os

In [2]:
def extract_document_embeddings(document):
    """
    Extract text embeddings for an input document in chunks and save the resulting vector db.
    """
    loader = PyPDFLoader(f'rag_docs/{document}')
    pages = loader.load_and_split()

    embeddings = AzureOpenAIEmbeddings(
        azure_deployment="MIMIC_text_embedding",
        openai_api_version="2024-02-15-preview",
    )

    faiss_index = FAISS.from_documents(pages, embeddings)
    faiss_index.save_local(f'data/{document[:-4]}_faiss_db')

In [3]:
def rag_generation(document,question):
    """
    Perform RAG generation for the question on the given document.
    """
    embeddings = AzureOpenAIEmbeddings(
            azure_deployment="MIMIC_text_embedding",
            openai_api_version="2024-02-15-preview",
        )
    faiss_index = FAISS.load_local(f'data/{document[:-4]}_faiss_db', embeddings,allow_dangerous_deserialization=True)
    
    docs = faiss_index.similarity_search(question)
    # Get the best matched page
    full_text = docs[0].page_content

    # Summarize the information
    prompt = f"""Using the following information below, you are asked to answer the user's question.

    {full_text}

    Answer:"""
    
    client = client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-15-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )
    
    response = client.chat.completions.create(
        model="MIMIC",
        messages=[
            {"role": "system", "content": f"{prompt}"},
            {"role": "user", "content": f"{question}"}
        ]
    )

    answer = response.choices[0].message.content

    return answer

In [4]:
extract_document_embeddings("FasterRisk.pdf")
answer = rag_generation("FasterRisk.pdf", "How is the risk score algorithm developed?")
print(answer)

The risk score algorithm is developed through an approach that efficiently produces a collection of high-quality risk scores learned from data. The algorithm uses a beam-search algorithm to generate a pool of almost-optimal sparse continuous solutions, each with a different support set. Then, each continuous solution is transformed into a separate risk score through a "star ray" search, where a range of multipliers are considered before rounding the coefficients sequentially to maintain low logistic loss. The algorithm returns all of these high-quality risk scores for the user to consider. This method completes within minutes and can be valuable in a broad variety of applications.
